# Import

In [1]:
import numpy as np
import pandas as pd
import random
import re

# Library Bantuan
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


# Library Sendiri
from libraryBantuan.katla import Katla

# Konstanta
PATH_FILE_TEXT = "D:\\Rama Nitip\\python\\katla\\file text\\"

# Baca Data

In [ ]:
daftar_kata = list(pd.read_csv(PATH_FILE_TEXT + 'lima huruf katla.csv', header=None)[0])
print(f"Banyak Kata : {len(daftar_kata)}")

Banyak Kata : 8314
